# Pretrained QCNN – Hackathon 2025

This notebook loads a pretrained Quantum Convolutional Neural Network (QCNN) trained on 20 BreastMNIST samples.

You can:
- Run inference on test samples
- Verify model predictions
- Modify test samples to assess performance

**Model Details:**
- Architecture: 6-qubit QCNN with 2 conv-pool layers + 1 dense SU(4)
- Total trainable parameters: 51
- Encoding: Amplitude
- Training: 1 repetition on 20 samples (balanced training set)

**Provided files:**
- `requirements.txt`: precise versions of necessary packages 
- `qcnn_pretrained_breastmnist_n2.npz`: pretrained weights
- `QCNN_inference_pyc.zip`: compiling circuit + computing probability
- `Pretrained_QCNN_Hackathon2025.ipynb`: this notebook


In [12]:
import sys

sys.path.append('./QCNN_inference_pyc.zip')

from QCNN_inference import compute_out, load_breastmnist_data


In [13]:
import numpy as np

# Load pretrained QCNN parameters
data = np.load("qcnn_pretrained_breastmnist_n20.npz")

# Extract the weights
w0 = data["weights_layer0"]      # Layer 0 (18 params)
w1 = data["weights_layer1"]      # Layer 1 (18 params; 6 dummy if skip_first_layer=True)
w_dense = data["dense_weights"]  # Final SU(4) layer (15 params)

weights = [w0, w1]
weights_last = w_dense


In [14]:
# Load test samples only (no training needed)
_, _, x_test, y_test = load_breastmnist_data(num_train=0, num_test=10, rng=np.random.default_rng(42))


100.0%


In [17]:
# Pick one test sample
sample = x_test[4]
label = y_test[4]

# Run the pretrained QCNN model on this sample
p = compute_out(weights, weights_last, [sample], [label])
print(f"Prediction probability: {p[0]}")
print(f"Predicted label: {int(p[0] > 0.5)}, Ground truth: {label}")


Prediction probability: 0.8359375
Predicted label: 1, Ground truth: 1


# The circuit is work as follow : 
-First layer is used for amplitude Embeddings wich convert classical data into quantum state this step could be depth expensive
-Second Layer performs quantum parametrized convolution as it use two local operation allowing to mimic the local convolution that yield good result in classical CNN, the locality of these gates can be use as parameter to detect more or less spread features.
-The third layer consist of forcing the model to compress the information, procedure known as pooling, as quantum unitary evolution is information preserving we need to make measurement to force the model give away some of the original image data. In the case of a well trained the discarded data is the one that is non explainatory of the outcome
-Then the layers alternates between convolutives block and pooling block to give more expresivity to the model.
